# OPAN5510 Lab Assignment - Joins

This lab focuses on using Polars to perform data joins and aggregations to answer business questions.

# Bike Trips Dataset

## Prerequisites

For this assignment, you'll need to use Polars for data manipulation.

*Insert a code block to import necessary packages (polars)*

In [1]:
# Import necessary packages
import polars as pl

## Load Datasets

Load the `bike_trips.csv` and `bike_weather.csv` files into Polars DataFrames called `trips` and `weather`, respectively.

*Insert code block that reads these files into Polars DataFrames*

In [2]:
# Load the bike trips and weather data
trips = pl.read_csv("https://raw.githubusercontent.com/philhetzel/opan5510-class9/refs/heads/main/data/bike_trips.csv", null_values='NA')
weather = pl.read_csv("https://raw.githubusercontent.com/philhetzel/opan5510-class9/refs/heads/main/data/bike_weather.csv", null_values=['NA',''])

trips = trips.with_columns(pl.col("date").str.strptime(pl.Date, "%Y-%m-%d").alias("date")) # ensure date column is typed correctly
weather = weather.with_columns(pl.col("date").str.strptime(pl.Date, "%Y-%m-%d").alias("date")) # ensure date column is typed correctly
print(trips.height)
print(weather.height)

78704
733


## Business Question 1: What was the average `duration` of trips that occurred in rainy weather?

#### Part A: Join the trips and weather data frames

The `trips` data represent every ride taken for a bike share company. The `weather` data represent the prevailing weather for a particular day. Join the `trips` and `weather` data together using the `date` column. Name the resulting DataFrame `trips_weather`.

*Insert a code block below that joins `trips` to `weather` using the `date` column.*

In [6]:
# Join trips and weather data
trips_weather = trips.join(weather, on="date", how="left")

# Show the first few rows
print(trips_weather.head())

shape: (5, 35)
┌──────┬──────────┬─────────────┬─────────────┬───┬─────────────┬────────┬────────────┬────────────┐
│ id   ┆ duration ┆ start_date  ┆ start_stati ┆ … ┆ cloud_cover ┆ events ┆ wind_dir_d ┆ zip_code_r │
│ ---  ┆ ---      ┆ ---         ┆ on_name     ┆   ┆ ---         ┆ ---    ┆ egrees     ┆ ight       │
│ i64  ┆ i64      ┆ str         ┆ ---         ┆   ┆ i64         ┆ str    ┆ ---        ┆ ---        │
│      ┆          ┆             ┆ str         ┆   ┆             ┆        ┆ i64        ┆ i64        │
╞══════╪══════════╪═════════════╪═════════════╪═══╪═════════════╪════════╪════════════╪════════════╡
│ 4721 ┆ 3        ┆ 2013-08-29T ┆ Market at   ┆ … ┆ 4           ┆ null   ┆ 286        ┆ 94107      │
│      ┆          ┆ 20:27:00Z   ┆ 10th        ┆   ┆             ┆        ┆            ┆            │
│ 4812 ┆ 3        ┆ 2013-08-29T ┆ 2nd at      ┆ … ┆ 4           ┆ null   ┆ 286        ┆ 94107      │
│      ┆          ┆ 21:30:00Z   ┆ Folsom      ┆   ┆             ┆        ┆  

#### Part B: Calculate the average `duration` of trips in poor weather

Using the `trips_weather` DataFrame, compute the average trip `duration` for days that had weather `events` (i.e. anytime a day had an event of anything but `null`). Name the column for average `duration`: `avg_duration`. The resulting DataFrame should have one row and be named `avg_bad_weather`.

*Insert a code block that shows the computation of the average trip duration for days that had a weather event.*

In [7]:
# Calculate average duration for trips with weather events
bad_weather_trips = trips_weather.filter(pl.col("events").is_not_null())
avg_bad_weather = bad_weather_trips.select(
    pl.col("duration").mean().alias("avg_duration")
)

print(avg_bad_weather)

shape: (1, 1)
┌──────────────┐
│ avg_duration │
│ ---          │
│ f64          │
╞══════════════╡
│ 10.34899     │
└──────────────┘


# Baseball Dataset

## Load Data

Load the `Batting.csv`, `People.csv`, and `HallOfFame.csv` datasets into Polars DataFrames.

*Insert a code block to load your datasets*

In [11]:
# Load baseball datasets
Batting = pl.read_csv("https://raw.githubusercontent.com/philhetzel/opan5510-class9/refs/heads/main/data/Batting.csv", null_values='NA')
People = pl.read_csv("https://raw.githubusercontent.com/philhetzel/opan5510-class9/refs/heads/main/data/People.csv", null_values='NA')
HallOfFame = pl.read_csv("https://raw.githubusercontent.com/philhetzel/opan5510-class9/refs/heads/main/data/HallOfFame.csv", null_values='NA')
Batting.head()


playerID,yearID,stint,teamID,lgID,G,AB,R,H,X2B,X3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
str,i64,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,str,str,str,i64
"""abercda01""",1871,1,"""TRO""",null,1,4,0,0,0,0,0,0,0,0,0,0,null,null,null,null,0
"""addybo01""",1871,1,"""RC1""",null,25,118,30,32,6,0,0,13,8,1,4,0,null,null,null,null,0
"""allisar01""",1871,1,"""CL1""",null,29,137,28,40,4,5,0,19,3,1,2,5,null,null,null,null,1
"""allisdo01""",1871,1,"""WS3""",null,27,133,28,44,10,2,2,27,1,1,0,2,null,null,null,null,0
"""ansonca01""",1871,1,"""RC1""",null,25,120,29,39,11,3,0,16,6,2,2,1,null,null,null,null,0


In [12]:
People.head()

playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,deathCountry,deathState,deathCity,nameFirst,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID,deathDate,birthDate
str,i64,i64,i64,str,str,str,i64,i64,i64,str,str,str,str,str,str,i64,i64,str,str,str,str,str,str,str,str
"""aardsda01""",1981,12,27,"""USA""","""CO""","""Denver""",null,null,null,null,null,null,"""David""","""Aardsma""","""David Allan""",215,75,"""R""","""R""","""2004-04-06""","""2015-08-23""","""aardd001""","""aardsda01""",null,"""1981-12-27"""
"""aaronha01""",1934,2,5,"""USA""","""AL""","""Mobile""",2021,1,22,"""USA""","""GA""","""Atlanta""","""Hank""","""Aaron""","""Henry Louis""",180,72,"""R""","""R""","""1954-04-13""","""1976-10-03""","""aaroh101""","""aaronha01""","""2021-01-22""","""1934-02-05"""
"""aaronto01""",1939,8,5,"""USA""","""AL""","""Mobile""",1984,8,16,"""USA""","""GA""","""Atlanta""","""Tommie""","""Aaron""","""Tommie Lee""",190,75,"""R""","""R""","""1962-04-10""","""1971-09-26""","""aarot101""","""aaronto01""","""1984-08-16""","""1939-08-05"""
"""aasedo01""",1954,9,8,"""USA""","""CA""","""Orange""",null,null,null,null,null,null,"""Don""","""Aase""","""Donald William""",190,75,"""R""","""R""","""1977-07-26""","""1990-10-03""","""aased001""","""aasedo01""",null,"""1954-09-08"""
"""abadan01""",1972,8,25,"""USA""","""FL""","""Palm Beach""",null,null,null,null,null,null,"""Andy""","""Abad""","""Fausto Andres""",184,73,"""L""","""L""","""2001-09-10""","""2006-04-13""","""abada001""","""abadan01""",null,"""1972-08-25"""


In [13]:
HallOfFame.head()

playerID,yearID,votedBy,ballots,needed,votes,inducted,category,needed_note
str,i64,str,i64,i64,i64,str,str,str
"""cobbty01""",1936,"""BBWAA""",226,170,222,"""Y""","""Player""",null
"""ruthba01""",1936,"""BBWAA""",226,170,215,"""Y""","""Player""",null
"""wagneho01""",1936,"""BBWAA""",226,170,215,"""Y""","""Player""",null
"""mathech01""",1936,"""BBWAA""",226,170,205,"""Y""","""Player""",null
"""johnswa01""",1936,"""BBWAA""",226,170,189,"""Y""","""Player""",null


## Question 2: How many home runs (`HR`) were hit by players born in Florida?

#### Part A: Join the `Batting` and `People` DataFrames together

In order to answer this question, you'll need to use the `Batting` and `People` DataFrames. The `Batting` DataFrame has hitting statistics for every season that an individual player has played. The `People` DataFrame represents biographical data about every professional baseball player.

Join the `People` DataFrame into the `Batting` DataFrame so that we can perform analysis on batting statistics using columns from the `People` DataFrame. Name the new DataFrame `stats_w_bio`.

*Insert a code block that joins the `Batting` DataFrame to the `People` DataFrame.*

In [14]:
# Join Batting and People DataFrames
stats_w_bio = Batting.join(People, on="playerID", how="left")

print(stats_w_bio.head())

shape: (5, 47)
┌───────────┬────────┬───────┬────────┬───┬──────────┬───────────┬────────────┬────────────┐
│ playerID  ┆ yearID ┆ stint ┆ teamID ┆ … ┆ retroID  ┆ bbrefID   ┆ deathDate  ┆ birthDate  │
│ ---       ┆ ---    ┆ ---   ┆ ---    ┆   ┆ ---      ┆ ---       ┆ ---        ┆ ---        │
│ str       ┆ i64    ┆ i64   ┆ str    ┆   ┆ str      ┆ str       ┆ str        ┆ str        │
╞═══════════╪════════╪═══════╪════════╪═══╪══════════╪═══════════╪════════════╪════════════╡
│ abercda01 ┆ 1871   ┆ 1     ┆ TRO    ┆ … ┆ aberd101 ┆ abercda01 ┆ 1939-11-11 ┆ 1850-01-02 │
│ addybo01  ┆ 1871   ┆ 1     ┆ RC1    ┆ … ┆ addyb101 ┆ addybo01  ┆ 1910-04-09 ┆ null       │
│ allisar01 ┆ 1871   ┆ 1     ┆ CL1    ┆ … ┆ allia101 ┆ allisar01 ┆ 1916-02-25 ┆ 1849-01-29 │
│ allisdo01 ┆ 1871   ┆ 1     ┆ WS3    ┆ … ┆ allid101 ┆ allisdo01 ┆ 1916-12-19 ┆ 1846-07-12 │
│ ansonca01 ┆ 1871   ┆ 1     ┆ RC1    ┆ … ┆ ansoc101 ┆ ansonca01 ┆ 1922-04-14 ┆ 1852-04-17 │
└───────────┴────────┴───────┴────────┴───┴──────────┴─

#### Part B: Calculate the total number of home runs that were hit by Florida-born players

You would like to perform an analysis about how many home runs (`HR`) were hit by players that were born in the `birthState` of Florida (`FL`). Using the new `stats_w_bio` DataFrame that combines batting statistics with biographical information, compute the total number of home runs (`HR`) hit by players who were born (`birthState`) in Florida (`FL`) and name the new column `total_hr`. The DataFrame should have one row and be named `florida_hr`.

*Insert a block of code that shows the computation of total home runs hit by players born in Florida. The output of this code block should be a DataFrame.*

In [15]:
# Calculate total home runs by Florida-born players
florida_players = stats_w_bio.filter(pl.col("birthState") == "FL")

# Compute total home runs
florida_hr = florida_players.select(
    pl.col("HR").sum().alias("total_hr")
)

print(florida_hr)

shape: (1, 1)
┌──────────┐
│ total_hr │
│ ---      │
│ i64      │
╞══════════╡
│ 16225    │
└──────────┘


## Question 3: What are the average number of career Hits (`H`) for Hall of Fame baseball players?

#### Part A: Clean the `HallOfFame` DataFrame

You would like to perform an analysis on the batting statistics of Hall of Fame baseball players. In order to answer this question, you'll need to use the `Batting` and `HallOfFame` DataFrames.

The first thing that you have to do to perform this analysis is to join the `HallOfFame` data into the `Batting` data to understand which players are "Hall of Famers". To join the data correctly, we have to ensure that the `playerID` field is unique in `HallOfFame`. Not every player in the `HallOfFame` DataFrame is a Hall of Famer; their `inducted` field must have the value of `Y` and their `category` field should have the value of `Player`. Name the new DataFrame `hall_inducted`.

*Insert a code block to transform the `HallOfFame` DataFrame to ensure that `playerID` is unique.*

In [16]:
# Clean HallOfFame DataFrame
hall_inducted = HallOfFame.filter(
    (pl.col("inducted") == "Y") & (pl.col("category") == "Player")
)

hall_inducted = hall_inducted.unique(subset=["playerID"])

print(hall_inducted.head())

shape: (5, 9)
┌───────────┬────────┬────────────┬─────────┬───┬───────┬──────────┬──────────┬─────────────┐
│ playerID  ┆ yearID ┆ votedBy    ┆ ballots ┆ … ┆ votes ┆ inducted ┆ category ┆ needed_note │
│ ---       ┆ ---    ┆ ---        ┆ ---     ┆   ┆ ---   ┆ ---      ┆ ---      ┆ ---         │
│ str       ┆ i64    ┆ str        ┆ i64     ┆   ┆ i64   ┆ str      ┆ str      ┆ str         │
╞═══════════╪════════╪════════════╪═════════╪═══╪═══════╪══════════╪══════════╪═════════════╡
│ delahed01 ┆ 1945   ┆ Old Timers ┆ null    ┆ … ┆ null  ┆ Y        ┆ Player   ┆ null        │
│ grimebu01 ┆ 1964   ┆ Veterans   ┆ null    ┆ … ┆ null  ┆ Y        ┆ Player   ┆ null        │
│ seaveto01 ┆ 1992   ┆ BBWAA      ┆ 430     ┆ … ┆ 425   ┆ Y        ┆ Player   ┆ null        │
│ riverma01 ┆ 2019   ┆ BBWAA      ┆ 425     ┆ … ┆ 425   ┆ Y        ┆ Player   ┆ null        │
│ wilheho01 ┆ 1985   ┆ BBWAA      ┆ 395     ┆ … ┆ 331   ┆ Y        ┆ Player   ┆ null        │
└───────────┴────────┴────────────┴─────────┴─

#### Part B: Find the average number of career hits across Hall of Fame players

Join the `hall_inducted` data into the `Batting` data by `playerID`. Aggregate the data to compute the average total hits (`H`) across all Hall of Fame players. To do this, you will first need to calculate the total number of hits for each player and then calculate the average hits across all players. Call the new column `average_hits`. The resulting DataFrame should have one row and be named `hof_hits`.

*Insert a code block that joins the `Batting` and the `hall_inducted` data together and then calculates the average number of career hits (`H`) across all Hall of Fame players. The output of this code block should be a DataFrame.*

In [18]:
# Calculate average career hits for Hall of Fame players
hof_batting = Batting.join(hall_inducted, on="playerID", how="inner")

# Calculate total hits per player
total_hits_per_player = hof_batting.group_by("playerID").agg(
    pl.col("H").sum().alias("total_hits")
)

# Calculate average hits across all Hall of Fame players
hof_hits = total_hits_per_player.select(
    pl.col("total_hits").mean().alias("average_hits")
)

print(hof_hits)

shape: (1, 1)
┌──────────────┐
│ average_hits │
│ ---          │
│ f64          │
╞══════════════╡
│ 1717.29918   │
└──────────────┘
